In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train0
from src.models.train_model import train1
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.manifold import TSNE

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0].to(device)

In [4]:
# triaing the model
gcn_model = GCN0(num_node_features=dataset.num_node_features,
                    num_classes=dataset.num_classes) \
                    .to(device)

gcn_model = train0(gcn_model, data)

In [5]:
data.y[data.train_mask].numpy().tolist().count(1)

20

In [6]:
from tqdm import tqdm

def reg_loss(p, t, train_mask, A_hat, mu=0.2, phi='ce'):
    """
    Regularization loss
    float tensor[N,C] p: predictions on the entire dataset
    int tensor[M] t: list of ground-truth class indeces for the training nodes
    bool tensor[N] train_mask: bool map for selecting the training nodes from the entire dataset
    bool tensor A_hat[N, N]: adjacency matrix for the entire dataset
    float mu: regularization factor
    """

    L_1 = F.cross_entropy(p[train_mask], t)

    if phi == 'ce':
        L_2 = F.cross_entropy(p, A_hat@p)
    elif phi == 'l2':
        L_2 = F.mse_loss(p, A_hat@p)
    elif phi == 'kld':
        L_2 = F.kl_div(p, A_hat@p)
    else:
        raise ValueError('phi must be one of ce (cross_entropy), l2 (squared error) or kld (kullback-leibler divergence)')

    
    M = train_mask.sum()
    N = train_mask.shape[0]

    return 1 / M * L_1 + mu / N * L_2

gcn_model = GCN0(num_node_features=dataset.num_node_features,
                 num_classes=dataset.num_classes) \
                .to(device)


def compute_a(data):
    a = torch.zeros(data.x.shape[0], data.x.shape[0])

    for i in data.edge_index.T:
        a[i[0], i[1]] = 1

    return a

A = compute_a(data)
D = torch.diag(A.sum(dim=0))
A_hat = D.inverse()@A

out = gcn_model(data)
print(out[data.train_mask].shape)

loss = reg_loss(
        p=out, # we make predictions on the entire dataset
        t=data.y[data.train_mask], # we assume to have ground-truth labels on only a subset of the dataset
        train_mask=data.train_mask, # hence, we also provide the train_mask, to know what nodes have labels
        A_hat=A_hat, 
        mu=0.1)

print(f'L_reg:\t{loss}')


torch.Size([140, 7])
L_reg:	0.012982075102627277


In [7]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(input)
print(target)
print(loss)
'''
.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
loss = F.cross_entropy(input, target)
loss.backward()
'''

tensor([[ 1.1461,  1.0924,  1.0553,  1.3307,  0.1736],
        [-0.8848,  1.7084,  0.6767,  0.6350,  1.3613],
        [ 0.4374, -0.5261, -2.4688,  1.1065, -0.7168]], requires_grad=True)
tensor([2, 1, 4])
tensor(1.6512, grad_fn=<NllLossBackward0>)


'\n.backward()\n# Example of target with class probabilities\ninput = torch.randn(3, 5, requires_grad=True)\ntarget = torch.randn(3, 5).softmax(dim=1)\nloss = F.cross_entropy(input, target)\nloss.backward()\n'